In [37]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import sklearn as sk
# import skforecast as skf
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile
from os import chdir

# Data Preprocessing

# Imputation if Necessary
from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator

# Data Preprocessing
# One Hot Encode 
from sklearn.preprocessing import OneHotEncoder

#Preprocessing (Scaling, Imputation, Scaling, Decomposition)
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.decomposition import PCA, TruncatedSVD

from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.compose import ColumnTransformer

# Models
# Custom, Simple to Complex
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Out of the Box
# from sktime.forecasting.fbprophet import Prophet
from prophet import Prophet

# Metrics
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape
# ++ root mean squared percentage error (rmspe)

from prophet.plot import plot_plotly, plot_components_plotly

from sklearn import set_config
set_config(transform_output="pandas")

data_folder = '../../data'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Rossman Store Sales Project

## Load Raw Dataset and create Interim Dataset

In [38]:
# Set data types for pandas to load csv
dtype={
                             'Store':'Int64'
                             ,'DayOfWeek':'Int64'
                             ,'Date':'str'
                             ,'Sales':'Int64'
                             ,'Customers':'Int64'
                             ,'Open':'Int64'
                             ,'Promo':'Int64'
                             ,'StateHoliday':'str'
                             ,'SchoolHoliday':'Int64'
                         }

data_train = pd.read_csv('../../data/raw/train.csv'
                         ,dtype=dtype
                         ,parse_dates=['Date']
)

data_test = pd.read_csv('../../data/raw/test.csv'
                        ,dtype=dtype
                         ,parse_dates=['Date']
                         ,index_col='Id'
)

sample_submission = pd.read_csv('../../data/raw/sample_submission.csv')
data_store = pd.read_csv('../../data/raw/store.csv')

# Dropping fields DayOfWeek (redundant), Customers (not needed), 'Open' filter is not needed as well since we only train and predict open days
# Filter necessary fields

data_train = data_train[data_train['Open']==1][['Store','Date','Promo','SchoolHoliday','Sales']]
data_test = data_test[data_test['Open']==1][['Store','Date','Promo','SchoolHoliday']]
data_train = data_train[['Store','Date','Promo','SchoolHoliday','Sales']]
data_test = data_test[['Store','Date','Promo','SchoolHoliday']]

data_train.rename({'Sales':'y','Date':'ds'},axis=1, inplace=True)
data_test.rename({'Sales':'y','Date':'ds'},axis=1, inplace=True)

data_train.set_index(['Store','ds'], inplace=True)
data_test.set_index(['Store','ds'], inplace=True)

data_train.index.levels[1].freq='D'
data_test.index.levels[1].freq='D'

data_train.sort_index(level=[0,1],inplace=True)
data_test.sort_index(level=[0,1],inplace=True)

data_train_x = data_train.drop('y',axis=1)
data_train_y = data_train['y']
data_test_x = data_test

In [39]:
data_train.dtypes, data_test.dtypes

(Promo            Int64
 SchoolHoliday    Int64
 y                Int64
 dtype: object,
 Promo            Int64
 SchoolHoliday    Int64
 dtype: object)

In [40]:
# Check for nulls
data_train.isna().sum(), data_test.isna().sum()

(Promo            0
 SchoolHoliday    0
 y                0
 dtype: int64,
 Promo            0
 SchoolHoliday    0
 dtype: int64)

In [41]:
# All entries appear once and there are no duplicates in train
(data_train.groupby(['Store','ds']).count()==1).mean()

Promo            1.0
SchoolHoliday    1.0
y                1.0
dtype: Float64

In [42]:
(data_test.groupby(['Store','ds']).count()==1).mean()

Promo            1.0
SchoolHoliday    1.0
dtype: Float64

In [43]:
# Data range is between Jan 2013 and July 2015
data_train.index.levels[1].min(), data_train.index.levels[1].max()

(Timestamp('2013-01-01 00:00:00'), Timestamp('2015-07-31 00:00:00'))

# PREPROCESSING

In [44]:
categorical_features = ["Promo","SchoolHoliday"]
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore", drop='if_binary', sparse_output=False))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_features),
    ],
    remainder="passthrough"
)

pipeline = Pipeline(steps=[
                    ('preprocessor',preprocessor)
                    ]
 )

## MODELLING

In [45]:
data_train_X = pipeline.fit_transform(X=data_train.drop('y',axis=1))
data_train_y = data_train[['y']]
data_test = pipeline.transform(X=data_test)

data_train = pd.concat([data_train_X,data_train_y],axis=1)

In [46]:
data_train

cat__Promo_1.0  cat__SchoolHoliday_1.0      y
Store ds                                                       
1     2013-01-02             0.0                     1.0   5530
      2013-01-03             0.0                     1.0   4327
      2013-01-04             0.0                     1.0   4486
      2013-01-05             0.0                     1.0   4997
      2013-01-07             1.0                     1.0   7176
...                          ...                     ...    ...
1115  2015-07-27             1.0                     1.0  10712
      2015-07-28             1.0                     1.0   8093
      2015-07-29             1.0                     1.0   7661
      2015-07-30             1.0                     1.0   8405
      2015-07-31             1.0                     1.0   8680

[844392 rows x 3 columns]

In [47]:
#  This can fit a single model
#  No hyperparameter optimization
p=Prophet()

yhat_train = p.fit(data_train.loc[1].reset_index()).predict(data_train.loc[1].reset_index())

yhat_test = p.predict(data_test.loc[1].reset_index())

yhat = pd.concat([yhat_train,yhat_test])

01:11:41 - cmdstanpy - INFO - Chain [1] start processing
01:11:41 - cmdstanpy - INFO - Chain [1] done processing


In [48]:
yhat_test.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2015-08-01,4674.826279,3541.817598,5666.400770,4674.826279,4674.826279,-112.074157,-112.074157,-112.074157,150.703020,150.703020,150.703020,-262.777177,-262.777177,-262.777177,0.0,0.0,0.0,4562.752122
1,2015-08-03,4674.628293,3590.435583,5773.483966,4674.628293,4674.628293,67.967743,67.967743,67.967743,387.929073,387.929073,387.929073,-319.961330,-319.961330,-319.961330,0.0,0.0,0.0,4742.596036
2,2015-08-04,4674.529299,3101.937953,5309.409485,4674.529299,4674.529299,-450.197727,-450.197727,-450.197727,-102.669099,-102.669099,-102.669099,-347.528628,-347.528628,-347.528628,0.0,0.0,0.0,4224.331572
3,2015-08-05,4674.430306,3018.134493,5164.751189,4674.430306,4674.430306,-593.295516,-593.295516,-593.295516,-219.647348,-219.647348,-219.647348,-373.648168,-373.648168,-373.648168,0.0,0.0,0.0,4081.134790
4,2015-08-06,4674.331313,2874.224271,5084.225077,4674.331313,4674.331313,-706.896844,-706.896844,-706.896844,-309.089463,-309.089463,-309.089463,-397.807381,-397.807381,-397.807381,0.0,0.0,0.0,3967.434468


In [49]:
plot_plotly(p,yhat)

In [50]:
plot_components_plotly(p,yhat_test)

In [52]:
# Save processed output as parquet
data_train.to_parquet(data_folder+'/processed/df_train.parquet')
data_test.to_parquet(data_folder+'/processed/df_test_X.parquet')